## 합성 테스트 데이터셋 생성

In [32]:
import langchain
import ragas

print(f"LangChain Version: {langchain.__version__}")
print(f"Ragas Version: {ragas.__version__}")

LangChain Version: 0.2.17
Ragas Version: 0.1.19


In [33]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("./data.pdf")

# 문서 로딩
docs = loader.load()

# 문서의 페이지수
len(docs)

101

In [5]:
print(len(docs))

101


In [6]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [64]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
import time

# Docx2txtLoader로 변경
loader = Docx2txtLoader("../dataset2.docx")
docs = loader.load()

# 작품명을 기준으로 문서 분리 함수
def split_by_artwork_title(docs):
    split_docs = []
    total_docs = len(docs)

    for idx, doc in enumerate(docs):
        content = doc.page_content  # 수정: Document 객체의 내용을 가져옴
        # 작품명을 기준으로 분리
        artworks = content.split("\n\n작품명:")
        for art in artworks:
            lines = art.strip().split("\n")
            title = "알 수 없음"
            if lines and lines[0].startswith("작품명:"):
                title = lines[0].replace("작품명:", "").strip()
            split_docs.append(Document(page_content=art.strip(), metadata={"작품명": title}))

        # 진행률 출력
        progress = ((idx + 1) / total_docs) * 100
        print(f"🔄 진행률: {progress:.2f}% 완료")
        time.sleep(0.1)  # 출력이 너무 빠르지 않도록 잠시 대기

    return split_docs

# 작품명을 기준으로 분리
docs = split_by_artwork_title(docs)
print(f"📄 분리된 문서 수: {len(docs)}")


🔄 진행률: 50.00% 완료
🔄 진행률: 100.00% 완료
📄 분리된 문서 수: 2


In [7]:
print(type(docs[50]))


<class 'langchain_core.documents.base.Document'>


In [8]:
docs[0].metadata

{'source': './data.pdf',
 'file_path': './data.pdf',
 'page': 0,
 'total_pages': 101,
 'Producer': 'iLovePDF',
 'ModDate': 'D:20250112103612Z',
 'filename': './data.pdf'}

In [17]:

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [18]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

In [19]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

In [20]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 1}

In [21]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=1,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

Generating:   0%|          | 0/1 [00:00<?, ?it/s]                 [ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['KIM Kyujin', 'Ancient Chinese engineers', '1913', '1922', 'Glory']
[ragas.testset.evolutions.INFO] seed question generated: "What significant events related to the year 1922 are mentioned in the context?"
[ragas.testset.filters.DEBUG] filtered question: {'feedback': "The question asks about significant events related to the year 1922, but it refers to 'the context' without providing any specific details or information about what that context entails. This reliance on unspecified external information makes the question unclear and unanswerable for someone who does not have access to that context. To improve clarity and answerability, the question could either include a brief description of the context or specify the type of events being inquir

In [22]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the significance of landscape in KIM O...,[• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 / ...,The significance of landscape in KIM Okjin's a...,simple,"[{'source': './data.pdf', 'file_path': './data...",True


In [23]:
# DataFrame의 상위 5개 행 출력
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the significance of landscape in KIM O...,[• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 / ...,The significance of landscape in KIM Okjin's a...,simple,"[{'source': './data.pdf', 'file_path': './data...",True


In [20]:
# DataFrame을 CSV 파일로 저장
test_df.to_csv("./ragas_qa_0112.csv", index=False)

## RAGAS를 활용한 평가

In [21]:
!pip install -qU faiss-cpu ragas

In [34]:
import pandas as pd

df = pd.read_csv("tmp.csv")
df

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"남관의 '세느강변'(1968)은 파리 체류기 동안 제작된 작품으로, 세느강변의 야경...",single_turn,{},True,'세느강변' 작품에서 작가는 어떤 기법을 사용했나요?,남관은 청색의 농도 변화와 어두운 음영을 사용하여 서정적인 풍경을 표현했습니다.
1,"윤영자의 '작품'(1976)은 청동으로 제작되었으며, 리드미컬하고 유기적인 형태를 ...",single_turn,{},True,'작품'은 어떤 감정을 전달하나요?,윤영자의 '작품'은 리드미컬한 형태와 유기적 곡선을 통해 생명의 맥박과 따뜻한 정감...
2,최만린의 '일월 72-7'(1972)은 유기적인 형태를 통해 생명의 지속성과 우주의...,single_turn,{},True,'일월 72-7'에서 표현된 생명의 이미지는 무엇인가요?,최만린의 '일월 72-7'은 유기적인 형태를 통해 생명의 지속성과 우주의 성장을 표...


In [48]:
test_dataset = test_dataset.add_column("question", ["'세느강변' 작품에서 작가는 어떤 기법을 사용했나요?", "'작품'은 어떤 감정을 전달하나요?", "'일월 72-7'에서 표현된 생명의 이미지는 무엇인가요?"])  # 예시 데이터


In [35]:
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done
0,"남관의 '세느강변'(1968)은 파리 체류기 동안 제작된 작품으로, 세느강변의 야경...",single_turn,{},True
1,"윤영자의 '작품'(1976)은 청동으로 제작되었으며, 리드미컬하고 유기적인 형태를 ...",single_turn,{},True
2,최만린의 '일월 72-7'(1972)은 유기적인 형태를 통해 생명의 지속성과 우주의...,single_turn,{},True


In [36]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 3
})

In [38]:
def convert_to_list(example):
    # contexts가 문자열이면 리스트로 변환
    if not isinstance(example["contexts"], list):
        example["contexts"] = [example["contexts"]]
    return example

# map 적용
test_dataset = test_dataset.map(convert_to_list)


Map: 100%|██████████| 3/3 [00:00<00:00, 156.40 examples/s]


In [39]:
test_dataset[1]["contexts"]

["윤영자의 '작품'(1976)은 청동으로 제작되었으며, 리드미컬하고 유기적인 형태를 통해 생명의 맥박과 따뜻한 정감을 전달한다."]

In [103]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [104]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:39<00:00,  5.62s/it]


In [106]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda


In [40]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks. 
Answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with "The document does not contain this information." 
Provide detailed and comprehensive answers, always include the artwork number, and ensure all answers are written in Korean. 
All answers should be formatted using beautiful Markdown syntax to make the response visually appealing and easy to read. Use headings, bullet points, and bold or italic text where appropriate to enrich the response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [41]:
import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### \n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### \n\n{llm_output.strip()}\n\n"


In [42]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

# 2. FAISS 데이터베이스 로드
persist_directory = "../faiss_artworks_0114"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [43]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.5        # 다양성과 관련성 간 균형
    }
)


In [44]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap
# 단계 8: 체인(Chain) 생성
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)

In [45]:
response = chain.invoke({"question": "낙양성동도이화작품번호는?"})
print(response)

### 

**낙양성동도이화 작품 번호:**  **1364**




In [47]:
print(test_dataset.column_names)

['contexts', 'evolution_type', 'metadata', 'episode_done']


In [49]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

["'세느강변' 작품에서 작가는 어떤 기법을 사용했나요?",
 "'작품'은 어떤 감정을 전달하나요?",
 "'일월 72-7'에서 표현된 생명의 이미지는 무엇인가요?"]

In [17]:
import os

# LangChain Tracing 비활성화
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_ENDPOINT"] = ""


In [52]:
answer = chain.batch(batch_dataset)
answer[:3]

["### \n\n**'세느강변' (Artwork Number: 157)에서 남관 작가의 기법:**\n\n- **색채 처리 기법:**  \n  남관 작가는 이 작품에서 **청색의 농도를 변화시키고 어둠에 변화를 주어** 화면 전체를 단색조로 표현합니다. 이러한 미묘한 색채 조절은 고도의 기술을 필요로 하며, 깊은 서정적 감정을 불러일으킵니다.\n\n- **추상화 기법:**  \n  세느강의 야경 속 불빛과 그림자를 연상시키는 어두운 음영을 통해 **추상화된 풍경**을 구현합니다. 이는 서정적인 분위기를 더하며, 구체적인 묘사보다는 감성적인 표현에 초점을 맞춥니다.\n\n이러한 기법들은 남관 작가의 독특한 동양적 추상 화법을 잘 보여줍니다.\n\n",
 "### \n\n## '작품'의 감정적 특징\n\n제공된 문서에 나열된 각 작품은 다음과 같은 감정적 특징을 전달합니다:\n\n**1. 양수아의 <작품> (1962, 번호: 2660)**\n* **분위기:** 침울하고 우울한 분위기\n* **감정:** 전쟁 이후의 염세적이고 우울한 인생관을 반영하며, 내면의 고뇌나 불안을 표현합니다. 짙은 암갈색과 황토색의 대비, 그리고 흰색의 운동감은 작가의 심리적 갈등을 시각적으로 전달합니다.\n\n**2. 양광자의 <작품> (1984, 번호: 1991)**\n* **분위기:** 심각함과 구원에 대한 갈망이 공존하는 분위기\n* **감정:** 작품 속 인물의 변형된 모습과 색채의 강렬한 사용은 심각한 좌절감과 동시에 구원을 갈망하는 열망을 표현합니다. 작가의 이중 문화 경험이 반영되어 한국인도 서구인도 아닌 독특한 인간상을 보여줍니다.\n\n**3. 황규태의 <작품> (1989, 번호: 2956)**\n* **분위기:** 비판적이고 묵시록적인 분위기\n* **감정:** 현대 문명에 대한 비판적 시각을 전달하며, 형광물질이 풍기는 미래적이면서도 위협적인 분위기는 현대인의 삶에 대한 경고를 담고 있습니다.\n\n**4. 김홍자의 <작품> (1988, 번호: 2962)**\n* *

In [53]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

In [59]:
# 'answer' 컬럼을 'reference'로 복사
test_dataset = test_dataset.add_column("reference", test_dataset["answer"])


In [54]:
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'answer'],
    num_rows: 3
})

In [60]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 12/12 [00:21<00:00,  1.83s/it]


{'context_precision': 1.0000, 'faithfulness': 0.4630, 'answer_relevancy': 0.0000, 'context_recall': 0.1667}

In [148]:
result_df = result.to_pandas()
result_df.head()

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,'산천'이라는 제목의 작품이 갖는 의미는 무엇인가요?,[• 작품 번호: 172\n• 제작 연도: 1974\n• 크기: 36.5×20×20...,### \n\n# '산천' 작품의 의미\n\n문서에 명시적으로 '산천'이라는 제목의...,'산천'이라는 제목의 작품이 갖는 의미는 문맥에서 드러나지 않습니다.,0.0,0.666667,0.000000,0.0
1,춘향전의 맥락에서 조용승이 갖는 의미는 무엇인가요?,"[수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n서...",### \n\n## 조용승의 의미: 춘향전 맥락에서\n\n조용승은 **조선시대 전통...,조용승은 1937년 출간된 '춘향전'을 각색한 '춘향이야기'의 작가라는 점에서 춘향...,1.0,0.500000,0.833598,0.5
2,한국 근대 미술 운동은 20세기 문화 지형에 어떤 중요한 공헌을 했나요?,[작가라는 점에는 이견이 없다. 그의 작품세계는 1970년대 중반 이후부터 많은\n...,### \n\n## 한국 근대 미술 운동의 20세기 문화 지형에 대한 공헌\n\n한...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.,0.0,0.000000,0.887446,1.0
3,이순자 작가의 작품 '중첩 3번'의 의미는 무엇인가요?,[• 작품명: 해초 따는 여인 / 海草 따는 女人 / Woman Taking Sea...,### \n\n**작품 정보:**\n- **제목:** 중첩 작품3 \n- **작가...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.,0.0,0.470588,0.916761,1.0
4,심형구 작가와 '포즈'라는 작품 사이에 문체적인 연관성이 있나요?,"[수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n서...","### \n\n**문체적 연관성 분석**\n\n심형구 작가와 그의 작품 **""<포즈...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.,0.0,0.764706,0.000000,1.0


In [135]:
result_df.to_csv("ragas_evaluation_0113.csv", index=False)

In [149]:
result_df.loc[:, "context_precision":"context_recall"]

,context_precision,faithfulness,answer_relevancy,context_recall
0,0.0,0.666667,0.000000,0.0
1,1.0,0.500000,0.833598,0.5
2,0.0,0.000000,0.887446,1.0
3,0.0,0.470588,0.916761,1.0
4,0.0,0.764706,0.000000,1.0
5,0.0,0.666667,0.898201,1.0
6,1.0,0.047619,0.919782,0.5
7,0.0,0.000000,0.000000,1.0
8,0.5,0.500000,0.000000,1.0


## DeepL을 사용해 데이터셋 번역

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
import pandas as pd

df = pd.read_csv("./ragas_qa_0113.csv")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What specific movements or styles emerged in v...,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
1,What significant art exhibition took place in ...,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",The significant art exhibition that took place...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
2,What is the significance of the song 'Graceful...,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
3,What is the significance of Korean traditional...,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
4,What themes link 'Spring Dream' to modern lit?,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",The answer to given question is not present in...,reasoning,"[{'source': './data.pdf', 'file_path': './data...",True


In [38]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [39]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 9/9 [00:07<00:00,  1.27it/s]


In [40]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,What specific movements or styles emerged in v...,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,What significant art exhibition took place in ...,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",The significant art exhibition that took place...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,What is the significance of the song 'Graceful...,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,What is the significance of Korean traditional...,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,What themes link 'Spring Dream' to modern lit?,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",The answer to given question is not present in...,reasoning,"[{'source': './data.pdf', 'file_path': './data...",True,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [41]:
# question, ground_truth 열을 확인합니다.
df.loc[:, ["question", "ground_truth"]].head()

,question,ground_truth
0,What specific movements or styles emerged in v...,The answer to given question is not present in...
1,What significant art exhibition took place in ...,The significant art exhibition that took place...
2,What is the significance of the song 'Graceful...,The answer to given question is not present in...
3,What is the significance of Korean traditional...,The answer to given question is not present in...
4,What themes link 'Spring Dream' to modern lit?,The answer to given question is not present in...


In [42]:
# question_translated, ground_truth_translated 열을 확인합니다.
df.loc[:, ["question_translated", "ground_truth_translated"]].head()

,question_translated,ground_truth_translated
0,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [8]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",simple,"[{'source': './data.pdf', 'file_path': './data...",True,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",reasoning,"[{'source': './data.pdf', 'file_path': './data...",True,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [43]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("./ragas_art_translate_0113.csv", index=False)

## HuggingFace에 Dataset 업로드

In [11]:
from datasets import Dataset

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 확인
print(dataset)

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})


In [12]:
from datasets import Dataset
import os

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 이름 설정 (원하는 이름으로 변경하세요)
dataset_name = "charagas-test-dataseteeee/"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=True,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="korean_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/chaeeee/ragas-test-dataset-0113/commit/b385cf5414f11d3234616cc9fa9b13b98be0b00e', commit_message='Upload dataset', commit_description='', oid='b385cf5414f11d3234616cc9fa9b13b98be0b00e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/chaeeee/ragas-test-dataset-0113', endpoint='https://huggingface.co', repo_type='dataset', repo_id='chaeeee/ragas-test-dataset-0113'), pr_revision=None, pr_num=None)

## 대용량 pdf 200페이지씩 분할

In [91]:
from pypdf import PdfReader, PdfWriter

def split_pdf(file_path, pages_per_split=200):
    # PDF 파일 열기
    reader = PdfReader(file_path)
    total_pages = len(reader.pages)

    # 몇 개의 파일로 나눌지 계산
    for i in range(0, total_pages, pages_per_split):
        writer = PdfWriter()
        split_start = i
        split_end = min(i + pages_per_split, total_pages)

        # 지정된 페이지 범위를 새로운 PDF에 추가
        for page_num in range(split_start, split_end):
            writer.add_page(reader.pages[page_num])

        # 결과 파일 저장
        output_filename = f"{file_path[:-4]}_part_{i//pages_per_split + 1}.pdf"
        with open(output_filename, "wb") as output_file:
            writer.write(output_file)
        
        print(f"{output_filename} 저장 완료! ({split_start + 1} ~ {split_end} 페이지)")

# PDF 파일 경로와 분할 페이지 수 입력
split_pdf("example.pdf", 200)


Note: you may need to restart the kernel to use updated packages.


## 수제 Data로 테스트

In [24]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

In [30]:
from datasets import Dataset

# 데이터셋 생성
dataset = Dataset.from_list([
    {
        "question": "남관의 '세느강변' 작품의 특징은 무엇인가요?",
        "context": "남관의 '세느강변'(1968)은 파리의 세느강을 소재로 한 서정적인 추상화입니다.",
        "answer": "'세느강변'은 청색의 농도와 어두운 음영으로 서정미가 감도는 추상화된 풍경을 보여줍니다.",
        "retrieved_contexts": [
            "남관의 '세느강변'(1968)은 파리의 세느강을 소재로 한 서정적인 추상화입니다."
        ],
        "reference": "'세느강변'은 청색의 농도를 변화시키고 어둠에 변화를 주며 단색조로 보이게 한 서정적인 추상화입니다."
    },
    {
        "question": "윤영자의 1976년 작품 '작품'은 어떤 조각인가요?",
        "context": "윤영자의 '작품'(1976)은 청동으로 제작되었으며, 리드미컬한 형태를 통해 생명의 율동을 표현했습니다.",
        "answer": "'작품'은 리드미컬한 형태와 유기적인 형태로 생명의 율동과 따뜻한 정감을 표현한 조각입니다.",
        "retrieved_contexts": [
            "윤영자의 '작품'(1976)은 청동으로 제작되었으며, 리드미컬한 형태를 통해 생명의 율동을 표현했습니다."
        ],
        "reference": "'작품'(1976)은 청동으로 제작되었으며, 생명의 맥박과 율동, 따뜻한 정감을 표현한 조각입니다."
    },
    {
        "question": "최만린의 '일월 72-7' 작품은 어떤 의미를 담고 있나요?",
        "context": "최만린의 '일월 72-7'(1972)은 유기적인 형태가 반복적으로 상승하는 조형으로, 생명의 지속성과 성장의 이미지를 표현한 작품입니다.",
        "answer": "'일월 72-7'은 생명의 지속성과 성장을 유기적인 형태로 표현한 작품입니다.",
        "retrieved_contexts": [
            "최만린의 '일월 72-7'(1972)은 유기적인 형태가 반복적으로 상승하는 조형으로, 생명의 지속성과 성장의 이미지를 표현한 작품입니다."
        ],
        "reference": "'일월 72-7'은 유기적인 형태를 통해 생명의 성장과 지속성을 시각화한 작품입니다."
    }
])


In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)



In [27]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:24<00:00,  3.57s/it]


In [28]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda
/tmp/ipykernel_22807/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)


In [57]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

# 평가 실행
result = evaluate(
    dataset=dataset,
    llm=llm,  # 기존에 정의한 LLM 평가 모델
    embeddings=embedding_model,  # 기존에 정의한 임베딩 모델
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
    ],
)

print(result)


Evaluating:   0%|          | 0/12 [00:15<?, ?it/s]


KeyboardInterrupt: 

Exception raised in Job[5]: AssertionError(llm must be set to compute score)
Exception raised in Job[7]: AttributeError('NoneType' object has no attribute 'generate')
Exception raised in Job[0]: AttributeError('NoneType' object has no attribute 'generate')
